In [1]:
import os
os.chdir("../")
os.getcwd()

'c:\\Users\\Ansh Lulla\\VS-Code\\MLOps\\16-End-to-end-Fine-Tuning'

In [3]:
from srcc.textSummarizer.logging import logger
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer: Path

In [4]:
from srcc.textSummarizer.constants import *
from srcc.textSummarizer.utils.common import *

class ConfigurationManager:
    def __init__(self, config_path=CONFIG_FILE_PATH, params_path=PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        
        create_directories([self.config.artifacts_root])
    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer=config.tokenizer
        )

        return data_transformation_config

In [6]:
import os
from srcc.textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_from_disk

class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer)
    
    def convert_examples_to_features(self, example_batch):
        input_encodings = self.tokenizer(example_batch["dialogue"], max_length=1024, truncation=True)
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch["summary"], max_length=128, truncation=True)

        return {
            "input_ids": input_encodings["input_ids"],
            "attention_mask": input_encodings["attention_mask"],
            "labels": target_encodings["input_ids"]
        }
    
    def convert(self):
        dataset = load_from_disk(self.config.data_path)
        dataset_pt = dataset.map(self.convert_examples_to_features, batched=True)
        dataset_pt.save_to_disk(os.path.join(self.config.root_dir, "samsum_dataset"))

In [9]:
config = ConfigurationManager()
data_transformation_config = config.get_data_transformation_config()
data_transformation = DataTransformation(config=data_transformation_config)
data_transformation.convert()

[2025-06-29 19:39:19,093: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-06-29 19:39:19,094: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-29 19:39:19,096: INFO: common: Created Directory at: artifacts]
[2025-06-29 19:39:19,097: INFO: common: Created Directory at: artifacts/data_transformation]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]c:\Users\Ansh Lulla\VS-Code\MLOps\venv\Lib\site-packages\transformers\tokenization_utils_base.py:3951: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 77688.13 examples/s]
